In [30]:
#!/usr/bin/env python
# coding: utf-8

# ## Task 2

#This is a script for testing for neutrinos reconstruction

import sys
sys.path.append("/eos/home-m/acraplet/.local/lib/python2.7/site-packages")
import uproot 
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from lbn_modified3 import LBN, LBNLayer
import tensorflow as tf


#for some reason pylorentz is installed somewhere differently ?
sys.path.append("/eos/home-a/acraplet/.local/lib/python2.7/site-packages")
from pylorentz import Momentum4
from pylorentz import Vector4
from pylorentz import Position4

# loading the tree
tree = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_AllHiggs_et.root")["ntuple"]
#tree = uproot.open("/eos/user/d/dwinterb/SWAN_projects/Masters_CP/MVAFILE_GluGluHToTauTauUncorrelatedDecay_Filtered_tt_2018.root")["ntuple"]
print("\n Tree loaded\n")



 Tree loaded



In [31]:
# define what variables are to be read into the dataframe
momenta_features = [ "pi_E_1", "pi_px_1", "pi_py_1", "pi_pz_1", #leading charged pi 4-momentum
              "pi_E_2", "pi_px_2", "pi_py_2", "pi_pz_2", #subleading charged pi 4-momentum
              "pi0_E_1","pi0_px_1","pi0_py_1","pi0_pz_1", #leading neutral pi 4-momentum
              "pi0_E_2","pi0_px_2","pi0_py_2","pi0_pz_2", #subleading neutral pi 4-momentum
              #"gen_nu_p_1", "gen_nu_phi_1", "gen_nu_eta_1", #leading neutrino, gen level
              #"gen_nu_p_2", "gen_nu_phi_2", "gen_nu_eta_2" #subleading neutrino, gen level  
                ] 

other_features = [ #"ip_x_1", "ip_y_1", "ip_z_1",        #leading impact parameter
                   #"ip_x_2", "ip_y_2", "ip_z_2",        #subleading impact parameter
                   #"y_1_1", "y_1_2",
                   "gen_phitt"
                 ]    # ratios of energies

target = [ "met", "metx", "mety", "aco_angle_1", "aco_angle_6", "aco_angle_5", "aco_angle_7"
         ]  #acoplanarity angle
    
selectors = [ "tau_decay_mode_1","tau_decay_mode_2",
             "mva_dm_1","mva_dm_2","rand","wt_cp_ps","wt_cp_sm",
            ]

variables4=(momenta_features+other_features+target+selectors) #copying Kinglsey's way cause it is very clean
print('Check 1')

df4 = tree.pandas.df(variables4)

print('Check 1')

df4 = df4[
      (df4["tau_decay_mode_1"] == 1) 
    & (df4["tau_decay_mode_2"] == 1) 
    & (df4["mva_dm_1"] == 1) 
    & (df4["mva_dm_2"] == 1)
]

print(0.7*len(df4),'This is the length') #up to here we are fine

df_ps = df4[
      (df4["rand"]<df4["wt_cp_ps"]/2)     #a data frame only including the pseudoscalars
]

df_sm = df4[
      (df4["rand"]<df4["wt_cp_sm"]/2)     #data frame only including the scalars
]

print("panda Data frame created \n")

df4.head()

Check 1
Check 1
0.0 This is the length
panda Data frame created 



,pi_E_1,pi_px_1,pi_py_1,pi_pz_1,pi_E_2,pi_px_2,pi_py_2,pi_pz_2,pi0_E_1,pi0_px_1,...,aco_angle_6,aco_angle_5,aco_angle_7,tau_decay_mode_1,tau_decay_mode_2,mva_dm_1,mva_dm_2,rand,wt_cp_ps,wt_cp_sm
entry,,,,,,,,,,,,,,,,,,,,,


The above will be mega slow so we do not want to do it each time !

In [32]:
def cross_product(vector3_1,vector3_2):
    if len(vector3_1)!=3 or len(vector3_1)!=3:
        print('These are not 3D arrays !')
    x_perp_vector=vector3_1[1]*vector3_2[2]-vector3_1[2]*vector3_2[1]
    y_perp_vector=vector3_1[2]*vector3_2[0]-vector3_1[0]*vector3_2[2]
    z_perp_vector=vector3_1[0]*vector3_2[1]-vector3_1[1]*vector3_2[0]
    return np.array([x_perp_vector,y_perp_vector,z_perp_vector])

def dot_product(vector1,vector2):
    if len(vector1)!=len(vector2):
        raise Arrays_of_different_size
    prod=0
    for i in range(len(vector1)):
        prod=prod+vector1[i]*vector2[i]
    return prod


def norm(vector):
    if len(vector)!=3:
        print('This is only for a 3d vector')
    return np.sqrt(vector[0]**2+vector[1]**2+vector[2]**2)

In [35]:
#define the usefull 4 momenta

#neutrinos refs, in E, px, py, pz form
#nu_1 = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_1"])), df4["gen_nu_eta_1"], df4["gen_nu_phi_1"], df4["gen_nu_p_1"])
#nu_2 = Momentum4.m_eta_phi_p(np.zeros(len(df4["gen_nu_phi_2"])), df4["gen_nu_eta_2"], df4["gen_nu_phi_2"], df4["gen_nu_p_2"])



#Charged and neutral pion momenta
pi_1_4Mom = Momentum4(df4["pi_E_1"],df4["pi_px_1"],df4["pi_py_1"],df4["pi_pz_1"])
pi_2_4Mom = Momentum4(df4["pi_E_2"],df4["pi_px_2"],df4["pi_py_2"],df4["pi_pz_2"]) 

#Same for the pi0
pi0_1_4Mom = Momentum4(df4["pi0_E_1"],df4["pi0_px_1"],df4["pi0_py_1"],df4["pi0_pz_1"])
pi0_2_4Mom = Momentum4(df4["pi0_E_2"],df4["pi0_px_2"],df4["pi0_py_2"],df4["pi0_pz_2"])

tau_1_vis = pi_1_4Mom + pi0_1_4Mom
tau_2_vis = pi_2_4Mom + pi0_2_4Mom

#print(nu_1.p_x[:10])
#print(tau_1_vis.p_x[:10], ' x components \n')

#print(nu_1.p_y[:10])
#print(tau_1_vis.p_y[:10], ' y components \n')

#print(nu_1.p_z[:10])
#print(tau_1_vis.p_z[:10], ' z components \n')

print(tau_1_vis)


Momentum4(array([], dtype=float64), array([], dtype=float64), array([], dtype=float64), array([], dtype=float64))


In [79]:
#Check the colinear approximation ? can calculate the dot product between the visible decay products and the nus
check_x_1 = np.array(nu_1.p_x-tau_1_vis.p_x)#dot_product(tau_1_vis[1:]/norm(tau_1_vis[1:]), nu_1[1:]/norm(nu_1[1:])))
check_x_2 = np.array(nu_2.p_x-tau_2_vis.p_x)#dot_product(tau_2_vis[1:]/norm(tau_2_vis[1:]), nu_2[1:]/norm(nu_2[1:])))
check_y_1 = np.array(nu_1.p_y-tau_1_vis.p_y)
check_y_2 = np.array(nu_2.p_y-tau_2_vis.p_y)


check_z_1 = []
check_z_2 = []

for i in range (len(nu_1.p_z)):
    if nu_1.p_z[i] != 9999:
        check_z_1.append(nu_1.p_z[i]-tau_1_vis.p_z[i])
    if nu_2.p_z[i] != 9999:
        check_z_2.append(nu_2.p_z[i]-tau_2_vis.p_z[i])

check_z_1 = np.array(check_z_1)
check_z_2 = np.array(check_z_2)

plt.figure()
plt.hist(tau_1_vis.p_x - nu_1.p_x, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_px and nu_1_px\nmean=%.2f, std=%.2f'% (check_x_1.mean(),check_x_1.std()))
plt.hist(tau_2_vis.p_x - nu_2.p_x, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_px and nu_2_px\nmean=%.2f, std=%.2f'% (check_x_2.mean(),check_x_2.std()))
plt.hist(tau_1_vis.p_y - nu_1.p_y, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_py and nu_1_py\nmean=%.2f, std=%.2f'% (check_y_1.mean(),check_y_1.std()))
plt.hist(tau_2_vis.p_y - nu_2.p_y, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_py and nu_2_py\nmean=%.2f, std=%.2f'% (check_y_2.mean(),check_y_2.std()))
plt.hist(check_z_1, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_pz and nu_1_pz\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_z_1.mean(),check_z_1.std()))
plt.hist(check_z_2, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_pz and nu_2_pz\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_z_2.mean(),check_z_2.std()))


plt.title('Sanity check - difference between \nsum(pions) and neutrino momenta components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between momenta components', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-13,13)
plt.legend()
plt.savefig('Check_angles_remove')


#plt.show()

<ipython-input-79-bdb3496a4d4e>:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [88]:
check_E_1 = []#dot_product(tau_1_vis[1:]/norm(tau_1_vis[1:]), nu_1[1:]/norm(nu_1[1:])))
check_E_2 = []#dot_product(tau_2_vis[1:]/norm(tau_2_vis[1:]), nu_2[1:]/norm(nu_2[1:])))

for i in range (len(nu_1.e)):
    if nu_1.e[i]!=9999:
        check_E_1.append(nu_1.e[i]-tau_1_vis.e[i])
    if nu_2.e[i]!=9999:
        check_E_2.append(nu_2.e[i]-tau_2_vis.e[i])

check_E_1 = np.array(check_E_1)
check_E_2 = np.array(check_E_2)
plt.figure()
plt.hist(check_E_1, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_1_vis_E and nu_1_E\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_E_1.mean(),check_E_1.std()))
plt.hist(check_E_2, bins = 1000, alpha = 0.5, 
         label = 'Difference between tau_2_vis_E and nu_2_E\nREMOVE 9999 mean=%.2f, std=%.2f'% (check_E_2.mean(),check_E_2.std()))


plt.title('Sanity check - difference between \nsum(pions) and neutrino energy components', fontsize = 'xx-large', weight = 'bold')
plt.xlabel('Difference between energies', fontsize = 'x-large')
plt.ylabel('Frequency', fontsize = 'x-large')
plt.grid()
plt.xlim(-13,13)
plt.legend()
plt.savefig('Check_energies')

In [99]:
sum_nu = nu_1 + nu_2

met = np.array(df4["met"])

# plt.close()
# plt.close()
# plt.close()
# plt.close()

sum_energies=[]
met_ref=[]

for i in range (len(nu_1.e)):
    if nu_1.e[i]!=9999 and nu_2.e[i]!=9999:
        sum_energies.append(sum_nu.e[i])
        met_ref.append(met[i])

plt.figure()
plt.plot(met_ref, sum_energies, 'gx')
plt.xlabel("Missing transverse energy", fontsize = 'x-large')
plt.ylabel("Energy of the sum of the neutrinos", fontsize = 'x-large')
plt.title("Sanity check, removing 9999 \n gen level nu energies and met", fontsize = 'xx-large', weight = 'bold')
plt.grid()
plt.savefig('Compare_nu_E_to_met')

In [120]:
x_ref = np.array([0, 6])
x_ref2 = [6,0]

gen_phitt = np.array(df4["gen_phitt"][:2000])*2*np.pi/180+np.pi

plt.figure()
plt.plot(x_ref, x_ref, '--', label = 'x=y')
plt.plot(x_ref, x_ref2, '--', label = 'y=-x')

plt.hist2d(df4["aco_angle_1"], df4["gen_phitt"], bins=50)
#plt.plot(df4["aco_angle_1"][:2000], gen_phitt, 'bx', label='aco_angle_1')
#plt.plot(df4["aco_angle_5"][:100], gen_phitt, 'gx', label='aco_angle_5')
#plt.plot(df4["aco_angle_6"][:100], gen_phitt, 'rx', label='aco_angle_6')
#plt.plot(df4["aco_angle_7"][:100], gen_phitt, 'kx', label='aco_angle_7')
#plt.xlabel("aco_angles", fontsize = 'x-large')
#plt.ylabel("gen_phitt, (rad & shifted)", fontsize = 'x-large')
plt.title("Sanity check,\n gen_phitt against aco angles", fontsize = 'xx-large', weight = 'bold')
#plt.grid()
#plt.legend()
plt.savefig('Gen_phitt-aco_1')

<ipython-input-120-d78cf37700c1>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [111]:
plt.close()
plt.close()
plt.close()
plt.close()